### Análise de Dados

#### Setup


In [0]:
%pip install fastapi uvicorn pyspark requests

Python interpreter will be restarted.
  Using cached fastapi-0.115.2-py3-none-any.whl (94 kB)
  Using cached uvicorn-0.32.0-py3-none-any.whl (63 kB)
  Using cached pyspark-3.5.3-py2.py3-none-any.whl
  Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached starlette-0.40.0-py3-none-any.whl (73 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Using cached pydantic_core-2.23.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Using cached anyio-4.6.2.post1-py3-none-any.whl (90 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached exceptiongroup-1.2.2-py3-none-any.whl (16 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/s

#### Realização de Requisição

In [0]:
import requests
# URL que você deseja acessar
url = "http://127.0.0.1:8000/extrair_dados"

# Fazendo a requisição GET
response = requests.get(url)

# Verificando o status da resposta
if response.status_code == 200:
    # A requisição foi bem-sucedida, você pode acessar o conteúdo
    data = response.json()  # Supondo que a resposta seja em JSON
    print(data)
else:
    # A requisição falhou, você pode tratar o erro
    print(f"Erro ao acessar a URL: {response.status_code}")

{'message': 'Arquivo Parquet não existia. Criado novo arquivo.'}


#### Análises Solicitadas

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, year, month, dayofmonth, lower

# Configurando Spark
spark = SparkSession.builder.appName("TransformacoesDados").getOrCreate()

def transform_data(parquet_path):
    """
    Função para ler dados do Parquet e aplicar transformações.
    """
    # Ler o arquivo Parquet
    df = spark.read.parquet(parquet_path)

    # 4.1 - Quantidade de notícias por ano, mês e dia de publicação
    df_data = df.groupBy(
        year("publishedAt").alias("ano"),
        month("publishedAt").alias("mes"),
        dayofmonth("publishedAt").alias("dia")
    ).agg(count("*").alias("quantidade_noticias"))

    df_data.write.mode("overwrite").parquet("/dbfs/FileStore/quantidade_noticias_por_data.parquet")

    # 4.2 - Quantidade de notícias por fonte e autor
    df_autor_fonte = df.groupBy(
        col("source.name").alias("fonte"),
        col("author").alias("autor")
    ).agg(count("*").alias("quantidade_noticias"))

    df_autor_fonte.write.mode("overwrite").parquet("/dbfs/FileStore/quantidade_noticias_por_fonte_autor.parquet")

    # 4.3 - Quantidade de aparições de 3 palavras chaves por ano, mês e dia
    palavras_chaves = ["genomics","genomic","genetic sequencing","DNA research","genetic analysis"]  # Altere para as suas palavras-chave

    # Filtrar o DataFrame com base nas palavras-chave
    df_keywords = df.filter(lower(col("title")).contains(palavras_chaves[0]) |
                            lower(col("title")).contains(palavras_chaves[1]) |
                            lower(col("title")).contains(palavras_chaves[2]))

    df_keywords_data = df_keywords.groupBy(
        year("publishedAt").alias("ano"),
        month("publishedAt").alias("mes"),
        dayofmonth("publishedAt").alias("dia")
    ).agg(count("*").alias("quantidade_aparicoes"))

    df_keywords_data.write.mode("overwrite").parquet("/dbfs/FileStore/quantidade_aparicoes_palavras_chaves.parquet")

    print("Dados transformados e armazenados com sucesso.")

# Definir o caminho do Parquet
PARQUET_PATH = "/dbfs/FileStore/dados.parquet"

# Executar a transformação
transform_data(PARQUET_PATH)


Dados transformados e armazenados com sucesso.


In [0]:
from pyspark.sql import SparkSession

# Configurando Spark
spark = SparkSession.builder.appName("LerDadosTransformados").getOrCreate()

def read_and_print_parquet(parquet_path):
    """
    Função para ler um arquivo Parquet e imprimir seus dados.
    """
    # Ler o arquivo Parquet
    df = spark.read.parquet(parquet_path)

    # Mostrar os dados
    df.show(truncate=False)
    print(f"Número de registros: {df.count()}")

# Definindo os caminhos dos novos arquivos Parquet
parquet_noticias_por_data = "/dbfs/FileStore/quantidade_noticias_por_data.parquet"
parquet_noticias_por_fonte_autor = "/dbfs/FileStore/quantidade_noticias_por_fonte_autor.parquet"
parquet_aparicoes_palavras_chaves = "/dbfs/FileStore/quantidade_aparicoes_palavras_chaves.parquet"

# Lendo e imprimindo os dados dos arquivos Parquet
print("Dados de quantidade de notícias por data:")
read_and_print_parquet(parquet_noticias_por_data)

print("\nDados de quantidade de notícias por fonte e autor:")
read_and_print_parquet(parquet_noticias_por_fonte_autor)

print("\nDados de aparições de palavras-chave por data:")
read_and_print_parquet(parquet_aparicoes_palavras_chaves)


Dados de quantidade de notícias por data:
+----+---+---+-------------------+
|ano |mes|dia|quantidade_noticias|
+----+---+---+-------------------+
|2024|9  |23 |2                  |
|2024|9  |21 |1                  |
|2024|9  |25 |2                  |
|2024|9  |26 |2                  |
|2024|10 |12 |1                  |
|2024|10 |11 |3                  |
|2024|10 |14 |3                  |
|2024|10 |15 |2                  |
|2024|10 |18 |5                  |
|2024|10 |10 |3                  |
|2024|10 |2  |2                  |
|2024|10 |8  |2                  |
|2024|10 |9  |1                  |
|2024|10 |17 |2                  |
|2024|10 |16 |1                  |
|2024|9  |29 |1                  |
|2024|10 |1  |1                  |
|2024|9  |30 |1                  |
+----+---+---+-------------------+

Número de registros: 18

Dados de quantidade de notícias por fonte e autor:
+------------------------+-------------------------------+-------------------+
|fonte                   |autor 